In [7]:
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp
import os
import numpy as np

import matplotlib.pyplot as plt
from src.models.ig_cnn import ig_cnn_model, get_cnn_dataset, InformationGainCallback, ImagesCallback

* [How connect spectrum](www.fourmilab.ch/documents/specrend/)
* [Python library](https://python-colormath.readthedocs.io/en/latest/color_objects.html#spectralcolor)

In [11]:
hparams = {
    'image_path' : '../data/raw/still_life/sample_22.mat',
    'model_def'  : 'C-25-3-sigmoid-valid,C-25-1-sigmoid-same',
    'input_size' : 3 ,
    'log_name' : 'C3'
}

In [12]:
# Parameters
logdir = os.path.join('..', 'logs', hparams['log_name'])


# Input Data
inputs, outputs, pad_output, nopad_input = get_cnn_dataset(hparams['image_path'],
                                                           input_size=hparams['input_size'])
# Model Definition
model = ig_cnn_model(hparams['model_def'])
model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['mean_squared_error'])

# Data split
training_set_size = 2500
perm = np.random.permutation(inputs.shape[0])
ins = inputs[perm[:training_set_size], :]
outs = outputs[perm[:training_set_size], :]

# TODO: Check log dir existence

callbacks = [tf.keras.callbacks.TensorBoard(log_dir=logdir),
             InformationGainCallback(input_image=nopad_input,
                                     target_image=pad_output,
                                     logdir=logdir),
             ImagesCallback(input_image=nopad_input,
                            target_image=pad_output,
                            logdir=logdir),
             hp.KerasCallback(logdir,hparams),
             # NOTE: This callback has to be last
             tf.keras.callbacks.EarlyStopping(patience=6, min_delta=10 ** -5),
 ]

history = model.fit(ins,
                    outs,
                    epochs=100,
                    validation_split=0.2,
                    verbose=1,
                    callbacks=callbacks
                    )


Epoch 1/100
63/63 [==============================] - 0s 7ms/step - loss: 0.0057 - mean_squared_error: 0.0057 - val_loss: 0.0060 - val_mean_squared_error: 0.0060
Epoch 2/100
63/63 [==============================] - 0s 5ms/step - loss: 0.0044 - mean_squared_error: 0.0044 - val_loss: 0.0048 - val_mean_squared_error: 0.0048
Epoch 3/100
63/63 [==============================] - 0s 5ms/step - loss: 0.0039 - mean_squared_error: 0.0039 - val_loss: 0.0045 - val_mean_squared_error: 0.0045
Epoch 4/100
63/63 [==============================] - 0s 5ms/step - loss: 0.0037 - mean_squared_error: 0.0037 - val_loss: 0.0045 - val_mean_squared_error: 0.0045
Epoch 5/100
63/63 [==============================] - 0s 5ms/step - loss: 0.0037 - mean_squared_error: 0.0037 - val_loss: 0.0043 - val_mean_squared_error: 0.0043
Epoch 6/100
63/